In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/Users/vithurshan/Desktop/VizWizSmall/server_downloads/DCM/Batch8/ModelOutput.txt', header=None)

In [5]:
df

,0
0,errD: 2.05
1,g_loss: 15.54 w_loss: 12.04 s_loss: 15.29 feat...
2,[0/600][187]
3,Loss_D: 2.29 Loss_C: 40.81 T...
4,Save C/D models.
...,...
3561,g_loss: 11.83 w_loss: 3.31 s_loss: 6.93 featur...
3562,[599/600][187]
3563,Loss_D: 1.48 Loss_C: 21.84 T...
3564,Save C/D models.


In [6]:
def get_required_data(df):
    dicts = {}
    keys = range(599)
    for i in keys:
        x = []
        for index, row in df.iterrows():
            row_value = row[0]
            if 'Loss_D' in row_value:
                x.append(row_value)
                df.drop(index, inplace=True)
                break
            else:
                df.drop(index, inplace=True)
        dicts[i] = x

    new_df = pd.DataFrame(dicts.items())
    new_df.rename(columns={0: 'Epoch', 1: 'Results'}, inplace=True)
    return new_df

def split_data(df):
    df['Results'] = df['Results'].apply(lambda x: x[0].split(' '))
    df['Results'] = df['Results'].apply(lambda x: list(filter(None, x)))
    return df

def final_output_txt(df):
    df['Loss_D'] = df['Results'].apply(lambda x: x[1])
    df['Loss_C'] = df['Results'].apply(lambda x: x[3])
    df['Time'] = df['Results'].apply(lambda x: x[5])
    df.drop(['Results'], axis=1, inplace=True)
    return df        

In [7]:
new_df = get_required_data(df)
split_df = split_data(new_df)
final_df = final_output_txt(split_df)    

In [8]:
final_df

,Epoch,Loss_D,Loss_C,Time
0,0,2.29,40.81,132.81s
1,1,1.80,36.23,121.88s
2,2,2.16,30.80,122.78s
3,3,2.14,53.85,123.03s
4,4,2.01,43.43,123.68s
...,...,...,...,...
594,594,1.97,23.87,136.15s
595,595,1.46,30.82,139.35s
596,596,1.44,25.40,138.29s
597,597,1.54,18.39,138.06s


In [9]:
final_df.to_csv(r'/Users/vithurshan/Desktop/VizWizSmall/server_downloads/DCM/Batch8/results.txt', header=None, index=None, sep=' ', mode='a')

# Part 2

In [10]:
df = pd.read_csv('/Users/vithurshan/Desktop/VizWizSmall/server_downloads/DCM/Batch8/ModelOutput.txt',header=None)
df

,0
0,errD: 2.05
1,g_loss: 15.54 w_loss: 12.04 s_loss: 15.29 feat...
2,[0/600][187]
3,Loss_D: 2.29 Loss_C: 40.81 T...
4,Save C/D models.
...,...
3561,g_loss: 11.83 w_loss: 3.31 s_loss: 6.93 featur...
3562,[599/600][187]
3563,Loss_D: 1.48 Loss_C: 21.84 T...
3564,Save C/D models.


In [11]:
def explodeData(data, word):
    dicts = {}
    x = []
    key = 0
    for index, row in df.iterrows():
        row_value = row[0]
        if word in row_value:
            x.append(row_value)
            data.drop(index, inplace=True)
        elif '[' in row_value:
            dicts[key] = x
            x = []
            key = key + 1
            data.drop(index, inplace=True)
        else:
            data.drop(index, inplace=True)
    new_df = pd.DataFrame(dicts.items())
    new_df = new_df.explode(1)
    return new_df

In [12]:
new_df = explodeData(df, 'errD')

In [13]:
new_df['vwDloss'] = new_df[1].apply(lambda x: x.split(':')[1])

In [14]:
new_df.drop([0,1], axis=1, inplace=True)

In [15]:
new_df

,vwDloss
0,2.05
1,2.19
1,2.04
2,2.33
2,2.01
...,...
597,1.46
598,1.43
598,1.42
599,1.54


In [16]:
df = pd.read_csv('/Users/vithurshan/Desktop/VizWizSmall/server_downloads/DCM/Batch8/ModelOutput.txt',header=None)
new_df1 = explodeData(df, 'g_loss')

In [17]:
new_df1['g_loss'] = new_df1[1].apply(lambda x: x.split(' ')[1])
new_df1['w_loss'] = new_df1[1].apply(lambda x: x.split(' ')[3])
new_df1['s_loss'] = new_df1[1].apply(lambda x: x.split(' ')[5])
new_df1['feature_loss'] = new_df1[1].apply(lambda x: x.split(' ')[7])

In [18]:
new_df1.drop([0,1], axis=1, inplace=True)

In [19]:
new_df1

,g_loss,w_loss,s_loss,feature_loss
0,15.54,12.04,15.29,-0.56
1,13.56,15.70,17.33,-0.54
1,12.86,18.87,16.67,-0.57
2,2.96,9.96,14.44,-0.54
2,11.54,7.18,11.57,-0.62
...,...,...,...,...
597,15.61,6.62,9.87,-0.32
598,14.72,3.59,8.06,-0.26
598,9.63,7.24,10.00,-0.29
599,11.90,9.05,10.21,-0.35


In [20]:
result = pd.concat([new_df, new_df1], axis=1)

In [21]:
result['epoch'] = result.index

In [22]:
result

,vwDloss,g_loss,w_loss,s_loss,feature_loss,epoch
0,2.05,15.54,12.04,15.29,-0.56,0
1,2.19,13.56,15.70,17.33,-0.54,1
1,2.04,12.86,18.87,16.67,-0.57,1
2,2.33,2.96,9.96,14.44,-0.54,2
2,2.01,11.54,7.18,11.57,-0.62,2
...,...,...,...,...,...,...
597,1.46,15.61,6.62,9.87,-0.32,597
598,1.43,14.72,3.59,8.06,-0.26,598
598,1.42,9.63,7.24,10.00,-0.29,598
599,1.54,11.90,9.05,10.21,-0.35,599


In [23]:
result.dtypes

vwDloss         object
g_loss          object
w_loss          object
s_loss          object
feature_loss    object
epoch            int64
dtype: object

In [24]:
result['feature_loss'] = result['feature_loss'].astype(str).astype(float)
result['s_loss'] = result['s_loss'].astype(str).astype(float)
result['w_loss'] = result['w_loss'].astype(str).astype(float)
result['g_loss'] = result['g_loss'].astype(str).astype(float)
result['vwDloss'] = result['vwDloss'].astype(str).astype(float)

In [25]:
result = result.groupby(by=['epoch']).mean()

In [26]:
result

,vwDloss,g_loss,w_loss,s_loss,feature_loss
epoch,,,,,
0,2.050,15.540,12.040,15.290,-0.560
1,2.115,13.210,17.285,17.000,-0.555
2,2.170,7.250,8.570,13.005,-0.580
3,2.580,6.230,16.795,12.095,-0.560
4,2.085,14.770,15.810,15.580,-0.605
...,...,...,...,...,...
595,1.700,20.895,7.240,8.000,-0.325
596,1.495,9.195,6.315,8.220,-0.280
597,2.085,10.450,6.270,10.465,-0.280


In [27]:
result.to_csv(r'/Users/vithurshan/Desktop/VizWizSmall/server_downloads/DCM/Batch8/OtherResults.txt', header=None, index=None, sep=' ', mode='a')